In [23]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import numpy as np
from functools import partial
import visualization_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [25]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix,MeshcatVisualizer, StartMeshcat
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
from pydrake.all import PointCloud

import time

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath


In [26]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [27]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0.0
x_fac = 0.5
locs = [ [0,0,0], 
        [x_fac, 1.4*sp,0.4], [x_fac,-1.4*sp,0.4], [-x_fac,-1.4*sp,0.4], [-x_fac,1.4,0.4], 
        [0.0 ,0 , 0.95], [0.0 ,0 , -0.05]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1


plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

meshcat = StartMeshcat()
meshcat.Delete()
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat)
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
diagram.ForcedPublish(diagram_context)

DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_prm_certify.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_prm_certify.runfiles/drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projec

In [28]:
#compute limits in s-space
limits_s = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))
limits_s = np.array(limits_s)

In [29]:
# The object we will use to perform our certification.
t0 = time.time()
cspace_free_path = CspaceFreePath(plant, scene_graph, q_star, maximum_path_degree = 1, plane_order = 1)
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

Time to construct line certifier = 8.113078355789185s


In [30]:
from pydrake.all import InverseKinematics
ik = InverseKinematics(plant, diagram.GetMutableSubsystemContext(plant, diagram_context))
min_dist = 1e-5
collision_constraint = ik.AddMinimumDistanceConstraint(
            min_dist, 1e-5)

def check_collision_q_by_ik(q, min_dist=1e-5):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        return 1 - 1 * \
            float(collision_constraint.evaluator().CheckSatisfied(q, min_dist))
    else:
        return 1
def check_collision_s_by_ik(s, min_dist=1e-5):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_ik(q, min_dist)

In [31]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]



def collision(pos, col_func_handle):
    return col_func_handle(pos)

def collision_bad(pos, col_func_handle):
    return 1-col_func_handle(pos)

prm_col_fn_handle = partial(collision, col_func_handle = check_collision_s_by_ik)
prm_col_fn_handle_bad = partial(collision_bad, col_func_handle = check_collision_s_by_ik)

PRM = prm.PRM( 
            limits_s,
            num_points = 1000,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 3, 
            dist_thresh = 10,
            num_col_checks = 10,
            max_it = 1e3,
            verbose = True,
            plotcallback = None
            )

# PRM_bad = prm.PRM( 
#             limits_t,
#             num_points = 2000,
#             col_func_handle = prm_col_fn_handle_bad,
#             num_neighbours = 5, 
#             dist_thresh = .5,
#             num_col_checks = 10,
#             verbose = True,
#             plotcallback = None
#             )

# PRM.add_start_end(start, target)
# PRM.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])
print(tot_num_edges)

[PRM] Samples 0
[PRM] Samples 30
[PRM] Samples 60
[PRM] Samples 90
[PRM] Samples 120
[PRM] Samples 150
[PRM] Samples 180
[PRM] Samples 210
[PRM] Samples 240
[PRM] Samples 270
[PRM] Samples 300
[PRM] Samples 330
[PRM] Samples 360
[PRM] Samples 390
[PRM] Samples 420
[PRM] Samples 450
[PRM] Samples 480
[PRM] Samples 510
[PRM] Samples 540
[PRM] Samples 570
[PRM] Samples 600
[PRM] Samples 630
[PRM] Samples 660
[PRM] Samples 690
[PRM] Samples 720
[PRM] Samples 750
[PRM] Samples 780
[PRM] Samples 810
[PRM] Samples 840
[PRM] Samples 870
[PRM] Samples 900
[PRM] Samples 930
[PRM] Samples 960
[PRM] Samples 990
[PRM] Nodes connected: 0
[PRM] Nodes connected: 20
[PRM] Nodes connected: 40
[PRM] Nodes connected: 60
[PRM] Nodes connected: 80
[PRM] Nodes connected: 100
[PRM] Nodes connected: 120
[PRM] Nodes connected: 140
[PRM] Nodes connected: 160
[PRM] Nodes connected: 180
[PRM] Nodes connected: 200
[PRM] Nodes connected: 220
[PRM] Nodes connected: 240
[PRM] Nodes connected: 260
[PRM] Nodes connected

DEBUG:drake:Meshcat connection closed from 
DEBUG:drake:Meshcat connection opened from 0000:0000:0000:0000:0000:0000:0000:0001


In [32]:
body = plant.GetBodyByName("iiwa_link_7")
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
def visualize_task_space_trajectory(segment_start, segment_end,
                                    body, color = Rgba(0,1,0,0.5),
                                    path_size = 0.01, num_points = 100, path = "prm/seg"):
    s_waypoints = np.linspace(segment_start, segment_end, num_points)
    points = []
    for s in s_waypoints:
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        points.append(plant.EvalBodyPoseInWorld(plant_context, body).translation())
    points = np.array(points)
    pc = PointCloud(len(points))
    pc.mutable_xyzs()[:] = points.T
    meshcat.SetObject(path, pc, point_size = path_size, rgba=color)


def plot_prm(PRM, body, color = Rgba(0,1,0,0.5),
                                    path_size = 0.01, num_points = 1000, prefix = "prm"):
    meshcat.Delete(prefix)
    for idx, (s0, s1) in enumerate(PRM.prm_pairs):
        visualize_task_space_trajectory(s0, s1, body, color,
                                        path_size, num_points, 
                                        f"{prefix}/seg_{idx}")
plot_prm(PRM, body)


In [35]:
meshcat.Delete("prm")


In [13]:
def make_prm_line_pairs(PRM):
    endpoint_index_set = set()
    for neighbors in PRM.adjacency_list:
        for n in neighbors[1:]:
            endpoint_index_set.add((neighbors[0], n))
    s0 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    s1 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    for i, (idx0, idx1) in enumerate(endpoint_index_set):
        s0[i] = PRM.nodes[idx0]
        s1[i] = PRM.nodes[idx1]
    return s0, s1
s0_mat_safe, s1_mat_safe = make_prm_line_pairs(PRM)
# s0_mat_unsafe, s1_mat_unsafe = make_prm_line_pairs(PRM_bad)
print(s0_mat_safe.shape)
# print(s0_mat_unsafe.shape)

(10, 7)


## Set up the sliders so we can move the plant around manually

In [40]:
sliders = []
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [9]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(f"Number of pairs {len(pairs)}")


Number of pairs 42


In [10]:
def count_num_safe(safe_points_vect):
    num_safe = 0
    for val in safe_points_vect:
        if(val):
            num_safe += 1
    return num_safe
def make_certifier(use_affine = True, num_points = -1):
    t0 = time.time()
    plane_type = rational_forward_kinematics.SeparatingPlaneOrder.kAffine if use_affine \
                    else rational_forward_kinematics.SeparatingPlaneOrder.kConstant
    line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                    plane_type, 
                                    q_star,
                                    set(), VerificationOption()
                                   )
    t1 = time.time()
    print(f"Time to construct line certifier = {t1-t0}s")
    return line_certifier

def do_certify(line_certifier, num_points = -1):
    t0 = time.time()
    safe_points, planes = line_certifier.CertifyTangentConfigurationSpaceLines(s0_mat_safe[:num_points,:], s1_mat_safe[:num_points,:])
    t1 = time.time()
    print(f"Certification of safe PRM in {t1-t0}s")
    num_safe1 = count_num_safe(safe_points)
    print(f"Certified {num_safe1}/{len(safe_points)} as safe")
    print(f"Failed to certify {len(safe_points)-num_safe1}/{len(safe_points)} safe points as safe")
    
    return (safe_points, planes, num_safe1)

In [11]:
t0 = time.time()
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kAffine, 
                                q_star,
                                set(), VerificationOption()
                               )
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 0/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 1/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 2/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 3/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 4/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 5/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 6/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 7/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 8/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 9/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 10/67

INFO:drake:Done rational 87/672
INFO:drake:numerator poly constructed in 0.353 s. Has degree: 7
INFO:drake:Done rational 88/672
INFO:drake:numerator poly constructed in 0.012 s. Has degree: 7
INFO:drake:Done rational 89/672
INFO:drake:numerator poly constructed in 0.014 s. Has degree: 7
INFO:drake:Done rational 90/672
INFO:drake:numerator poly constructed in 0.013 s. Has degree: 7
INFO:drake:Done rational 91/672
INFO:drake:numerator poly constructed in 0.013 s. Has degree: 7
INFO:drake:Done rational 92/672
INFO:drake:numerator poly constructed in 0.012 s. Has degree: 7
INFO:drake:Done rational 93/672
INFO:drake:numerator poly constructed in 0.012 s. Has degree: 7
INFO:drake:Done rational 94/672
INFO:drake:numerator poly constructed in 0.013 s. Has degree: 7
INFO:drake:Done rational 95/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 96/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 97/672
INFO:drake:numer

INFO:drake:Done rational 174/672
INFO:drake:numerator poly constructed in 0.009 s. Has degree: 7
INFO:drake:Done rational 175/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 176/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 177/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 178/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 179/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 180/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 181/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 182/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 183/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 1
INFO:drake:Done rational 184/672
INFO:drake:numerator poly constru

INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 263/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 264/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 265/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 266/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 267/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 268/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 269/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 270/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 271/672
INFO:drake:numerator poly constructed in 0.188 s. Has degree: 7
INFO:drake:Done rational 272/672
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
IN

INFO:drake:Done rational 350/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 351/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 352/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 353/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 354/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 355/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 356/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 357/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 358/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 359/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 360/672
INFO:drake:numerator poly constructed

INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 439/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 440/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 441/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 442/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 443/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 444/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 445/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 446/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 447/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake:Done rational 448/672
INFO:drake:numerator poly constructed in 0 s. Has degree: 5
INFO:drake

INFO:drake:numerator poly constructed in 0 s. Has degree: 3
INFO:drake:Done rational 527/672
INFO:drake:numerator poly constructed in 0.101 s. Has degree: 7
INFO:drake:Done rational 528/672
INFO:drake:numerator poly constructed in 0.008 s. Has degree: 7
INFO:drake:Done rational 529/672
INFO:drake:numerator poly constructed in 0.008 s. Has degree: 7
INFO:drake:Done rational 530/672
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:Done rational 531/672
INFO:drake:numerator poly constructed in 0.008 s. Has degree: 7
INFO:drake:Done rational 532/672
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:Done rational 533/672
INFO:drake:numerator poly constructed in 0.008 s. Has degree: 7
INFO:drake:Done rational 534/672
INFO:drake:numerator poly constructed in 0.006 s. Has degree: 7
INFO:drake:Done rational 535/672
INFO:drake:numerator poly constructed in 0.001 s. Has degree: 5
INFO:drake:Done rational 536/672
INFO:drake:numerator poly construc

INFO:drake:numerator poly constructed in 0.009 s. Has degree: 7
INFO:drake:Done rational 612/672
INFO:drake:numerator poly constructed in 0.01 s. Has degree: 7
INFO:drake:Done rational 613/672
INFO:drake:numerator poly constructed in 0.01 s. Has degree: 7
INFO:drake:Done rational 614/672
INFO:drake:numerator poly constructed in 0.01 s. Has degree: 7
INFO:drake:Done rational 615/672
INFO:drake:numerator poly constructed in 0.01 s. Has degree: 7
INFO:drake:Done rational 616/672
INFO:drake:numerator poly constructed in 0.01 s. Has degree: 7
INFO:drake:Done rational 617/672
INFO:drake:numerator poly constructed in 0.008 s. Has degree: 7
INFO:drake:Done rational 618/672
INFO:drake:numerator poly constructed in 0.01 s. Has degree: 7
INFO:drake:Done rational 619/672
INFO:drake:numerator poly constructed in 0.01 s. Has degree: 7
INFO:drake:Done rational 620/672
INFO:drake:numerator poly constructed in 0.009 s. Has degree: 7
INFO:drake:Done rational 621/672
INFO:drake:numerator poly constructed

Time to construct line certifier = 244.36708164215088s


In [14]:
i=0
t0 = time.time()
safe_points, planes = line_certifier.CertifyTangentConfigurationSpaceLine(s0_mat_safe[i,:], s0_mat_safe[i,:])
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")
print(f"Edge {i} is safe = {safe_points}")
print(f"Number of collision pairs is = {len(planes)}")

DEBUG:drake:Time to assign constraints is 39.066
DEBUG:drake:Program has 9100 variables and 5264 constraints.
DEBUG:drake:solvers::Solve will use Mosek
DEBUG:drake:Line
 s0: -7.82038
0.622673
 10.5504
0.549586
-2.21116
0.116868
-1.49121
 s1: -7.82038
0.622673
 10.5504
0.549586
-2.21116
0.116868
-1.49121
 certified in 0.184 s


Certification of safe PRM in 39.285932779312134s
Edge 0 is safe = True
Number of collision pairs is = 42
